<a href="https://colab.research.google.com/github/anjunjin/PFE-ING3-IA/blob/master/Nettoyages_des_tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

On peut Définir la seed pour que l'aléatoire soit consistant entre exécution

In [29]:
seed=42

Les pip install

In [30]:
%pip install --upgrade pandas

In [31]:
pip install emot --upgrade

In [32]:
import pickle
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')
DATA_PATH = "/content/drive/My Drive/PFE/"
with open(DATA_PATH+"data_tweets.txt", "rb") as file :  
    data_tweets = pickle.load(file)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
import emot
import re
import numpy as np

On retire les tweets parodiques (je ne sais pas si c'est toujours utile parce que juste en dessous je fais un truc équivalent)

In [34]:
data_tweets_sans_parodiques = data_tweets.copy()
data_tweets_sans_parodiques = data_tweets_sans_parodiques.drop(data_tweets_sans_parodiques[data_tweets_sans_parodiques.liability == 1].index)
data_tweets_sans_parodiques = data_tweets_sans_parodiques.reset_index(drop=True)

On sélectionne aléatoirement 10000 pour chaque catégories restantes

In [35]:
# Création des groupes à comparer
notfiables = [2] #Groupe des non fiables
df_notfiables = data_tweets[data_tweets['liability'].isin(notfiables)]
maybefiables = [3] #Groupe peut-être fiable
df_maybefiables = data_tweets[data_tweets['liability'].isin(maybefiables)]
isfiables = [4] #Groupe des fiables
df_isfiables = data_tweets[data_tweets['liability'].isin(isfiables)]

In [36]:
np.random.default_rng(seed)
#Faire un random seed dans random.choice pour conservé

#10000 tweets non fiables sélectionnés aléatoirement
remove_n = df_notfiables.shape[0]-10000
drop_indices = np.random.choice(df_notfiables.index, remove_n, replace=False)
df_notfiables = df_notfiables.drop(drop_indices)
#10000 tweets peut-être fiables sélectionnés aléatoirement
remove_n = df_maybefiables.shape[0]-10000
drop_indices = np.random.choice(df_maybefiables.index, remove_n, replace=False)
df_maybefiables = df_maybefiables.drop(drop_indices)
#10000 tweets fiables sélectionnés aléatoirement
remove_n = df_isfiables.shape[0]-10000
drop_indices = np.random.choice(df_isfiables.index, remove_n, replace=False)
df_isfiables = df_isfiables.drop(drop_indices)

In [37]:
df_notfiables['liability'] = 1
df_maybefiables['liability'] = df_notfiables['liability'].iloc[0]
df_isfiables['liability'] = 0
df_notfiables['liability_label'] = 'site plutôt douteux'
df_maybefiables['liability_label'] = df_notfiables['liability_label'].iloc[0]
df_isfiables['liability_label'] = 'site plutôt fiable'

In [38]:
# df_group = pd.concat([df_notfiables, df_isfiables]) #fausse informations vs fiable
# df_group = pd.concat([df_maybefiables, df_isfiables]) #fiabilité douteuse vs fiable
# df_group = pd.concat([df_notfiables, df_maybefiables]) #fausse informations vs fiabilité douteuse; Attention, dans la suite du programme les noms peuvent sembler bizarre pour cette ligne
df_group = pd.concat([df_notfiables.head(5000), df_maybefiables.head(5000), df_isfiables]) #faux+doute vs fiable

df_group = df_group.drop(columns=['user_id',
                                  'id_tweet',
                                  'created_at',
                                  'retweet_count',
                                  'favorite_count',
                                  'count_followers'
                                  ]
                         ).reset_index(drop=True)

df_group = df_group[['liability', 
                     'liability_label', 
                     'tweet', 
                     'description',
                     'user_screen_name'
                     ]]

In [39]:
df_group

,liability,liability_label,tweet,description,user_screen_name
0,1,site plutôt douteux,RT @DIVIZIO1: Recours déposé !,France-Soir était un journal fondé par Pierre ...,france_soir
1,1,site plutôt douteux,L’entretien complet : https://t.co/0EDJwyvqsM,France-Soir était un journal fondé par Pierre ...,france_soir
2,1,site plutôt douteux,"BEST OF - 🎙️ En Guadeloupe, Me Ellen Bessis et...",France-Soir était un journal fondé par Pierre ...,france_soir
3,1,site plutôt douteux,RT @CStrateges: Juste pour information…regarde...,France-Soir était un journal fondé par Pierre ...,france_soir
4,1,site plutôt douteux,🎙️Sylviane Noël s'était opposée frontalement a...,France-Soir était un journal fondé par Pierre ...,france_soir
...,...,...,...,...,...
19995,0,site plutôt fiable,Routinely adjusting your bike brakes will help...,Un site qui donne des conseils et astuces sur ...,wikiHow
19996,0,site plutôt fiable,#wikitips https://t.co/owfBGGtsns https://t.co...,Un site qui donne des conseils et astuces sur ...,wikiHow
19997,0,site plutôt fiable,Drink wine with the 4 S's in mind:\n1) See it....,Un site qui donne des conseils et astuces sur ...,wikiHow
19998,0,site plutôt fiable,"Rrrruuuuuunnnnnn, Forrreeesssst! https://t.co/...",Un site qui donne des conseils et astuces sur ...,wikiHow


Supression des tweets anglais et italiens. 

In [ ]:
! pip install stop-words

In [ ]:
#On récupère les stopwords italiens et anglais
from stop_words import get_stop_words
stop_words_en = get_stop_words('en')
stop_words_it = get_stop_words('it')

In [ ]:
#On enlève les mots italiens trop commun avec des mots français pour éviter la confusion
stop_words_it.remove('tu')
stop_words_it.remove('un')
stop_words_it.remove('lui')
stop_words_it.remove('ma')
stop_words_it.remove('non')
stop_words_it.remove('si')
stop_words_it.remove('se')
stop_words_it.remove('qui')
stop_words_it.remove('te')
stop_words_it.remove('sui')
stop_words_it.remove('c')
stop_words_it.remove('e')
stop_words_it.remove('i')
stop_words_it.remove('l')
stop_words_it.remove('o')
stop_words_it.remove('è')
stop_words_it.remove('a')
stop_words_it.remove('le')
stop_words_it.remove('la')
stop_words_it.remove('il')

In [ ]:
#Fonction pour détecter si la phrase est oui ou non italienne ou anglaise
def detect_language(tweet):
  tweet_toLow = tweet.lower()
  tweetList = tweet_toLow.split(" ")
  nbCommon_en = len(list(set(tweetList)&set(stop_words_en)))
  nbCommon_it = len(list(set(tweetList)&set(stop_words_it)))
  if nbCommon_en > 2 or nbCommon_it > 2:
    return True
  else: 
    return False
  

In [ ]:
#On supprime les éléments de la dataframe présentant des tweets anglais ou italiens
df_indexs= []
c = 0
for sentence in df_group['tweet']: 
  if detect_language(sentence):
      df_indexs.append(c)
  c = c+1
df_group = df_group.drop(labels=df_indexs, axis=0) 
df_group = df_group.reset_index(drop=True)

On sauvegarde le nettoyage des tweets dans un autre tableau

In [41]:
# df_tweets_clean = pd.DataFrame()
# df_tweets_clean['tweet'] = pd.DataFrame(df_group['tweet'])
df_tweets_clean = pd.DataFrame(df_group)

# reset index 
# df_tweets_clean.reset_index(drop=True)

# Mettre les mots en minuscule
# df_tweets_clean['minuscule'] = df_tweets_clean['tweet'].apply(lambda x : x.lower()) 
df_tweets_clean['tweet'] = df_tweets_clean['tweet'].apply(lambda x : x.lower()) 

In [42]:
# nettoyer hashtag #
# df_tweets_clean['nettoyer #'] = df_tweets_clean['minuscule'].apply(lambda x : re.sub(r"#\w+", "", x))
df_tweets_clean['tweet'] = df_tweets_clean['tweet'].apply(lambda x : re.sub(r"#\w+", "", x))

In [43]:
# nettoyer @
# df_tweets_clean['nettoyer @'] = df_tweets_clean['nettoyer #'].apply(lambda x : re.sub(r"@\w+", "", x))
df_tweets_clean['tweet'] = df_tweets_clean['tweet'].apply(lambda x : re.sub(r"@\w+", "", x))

In [44]:
# transformation des liens internet en "desident_link"
# df_tweets_clean['liens'] = df_tweets_clean['nettoyer @'].apply(lambda x : re.sub(r"https://t.co/[a-zA-Z\u00c0-\u00FF0-9]+", "desident_link", x))
df_tweets_clean['tweet'] = df_tweets_clean['tweet'].apply(lambda x : re.sub(r"https://t.co/[a-zA-Z\u00c0-\u00FF0-9]+", "desident_link", x))

In [45]:
# transformation des ! en "desident_exclamation"
# df_tweets_clean['nettoyer !'] = df_tweets_clean['liens'].apply(lambda x : re.sub(r"!", "desident_exclamation", x))
df_tweets_clean['tweet'] = df_tweets_clean['tweet'].apply(lambda x : re.sub(r"!", "desident_exclamation", x))

In [46]:
# transformation des ? en "desident_exclamation"
# df_tweets_clean['nettoyer ?'] = df_tweets_clean['nettoyer !'].apply(lambda x : re.sub(r"[?]", "desident_question", x))
df_tweets_clean['tweet'] = df_tweets_clean['tweet'].apply(lambda x : re.sub(r"[?]", "desident_question", x))

In [47]:
# fonction nettoyer ponctuation
import string
def Remove_Punctuation(text):
    result = "".join([ch for ch in text if ch not in string.punctuation])
    return result

In [48]:
# nettoyer les ponctuation
# df_tweets_clean['nettoyer ponctuaction'] = df_tweets_clean['nettoyer ?'].apply(lambda x: Remove_Punctuation(x))
df_tweets_clean['tweet'] = df_tweets_clean['tweet'].apply(lambda x: Remove_Punctuation(x))

Suppression des dates

In [ ]:
def replace_patterns(tweet):

patterns_jour = ['lundi', 'mardi', 'mercredi', 'jeudi', 'vendredi', 'samedi', 'dimanche']
patterns_mois = ['janvier', 'février', 'fevrier', 'mars', 'avril', 'mai', 'juin', 'juillet', 'août', 'aout', 'septembre', 'octobre', 'décembre', 'decembre']
patterns_date = patterns_jour+patterns_mois+
                ['matin', 'midi', 'soir', 'nuit',
                '202[0-9]']
# triple boucle pour faire des dates entières
for pattern in patterns_date:
  tweet = re.sub(pattern, 'desident_date', tweet)
return tweet

Suppression des émojis

In [49]:
# fonction (convert_emojis_to_text) : convertir les emojis en texte
# Entrée : text(un tweet)
# Sortie : text(après convertir)
def convert_emojis_to_text(text):
  # créer un objet : emot_obj 
  emot_obj = emot.core.emot()
  # les informations sont dans ce objet.emoji 
  emoji = emot_obj.emoji(text)
  # print('List emoji :     ', emoji) 
  
  # print('Text original :                 ', text)

  for i in range(len(emoji['value'])):
    text = re.sub(emoji['value'][i], "desident_emot_"+emoji['mean'][i], text)
    # Enlever les :
    text = re.sub(r':', '', text)

  # print('Text(convert emoji into text) : ', text)
  
  return text

In [50]:
# nettoyer les ponctuation
# df_tweets_clean['emoji'] = df_tweets_clean['nettoyer ponctuaction'].apply(lambda x: convert_emojis_to_text(x))
df_tweets_clean['tweet'] = df_tweets_clean['tweet'].apply(lambda x: convert_emojis_to_text(x))

In [53]:
df_group.head(10)

,liability,liability_label,tweet,description,user_screen_name
0,1,site plutôt douteux,rt recours déposé desidentexclamation,France-Soir était un journal fondé par Pierre ...,france_soir
1,1,site plutôt douteux,l’entretien complet desidentlink,France-Soir était un journal fondé par Pierre ...,france_soir
2,1,site plutôt douteux,best of desident_emot_studio_microphone️ en g...,France-Soir était un journal fondé par Pierre ...,france_soir
3,1,site plutôt douteux,rt juste pour information…regardez en détail ...,France-Soir était un journal fondé par Pierre ...,france_soir
4,1,site plutôt douteux,desident_emot_studio_microphone️sylviane noël ...,France-Soir était un journal fondé par Pierre ...,france_soir
5,1,site plutôt douteux,desident_emot_European_Union autant de dé...,France-Soir était un journal fondé par Pierre ...,france_soir
6,1,site plutôt douteux,desident_emot_microbelagitation autour du vari...,France-Soir était un journal fondé par Pierre ...,france_soir
7,1,site plutôt douteux,desident_emot_megaphonel’avortement n’est pas ...,France-Soir était un journal fondé par Pierre ...,france_soir
8,1,site plutôt douteux,desident_emot_Qatar pour la coupe du monde 202...,France-Soir était un journal fondé par Pierre ...,france_soir
9,1,site plutôt douteux,desident_emot_Netherlands bien que vivre dans ...,France-Soir était un journal fondé par Pierre ...,france_soir


In [52]:
df_tweets_clean.to_csv('Tweets_nettoyés.csv')
!cp Tweets_nettoyés.csv "/content/drive/My Drive/PFE"